<a href="https://colab.research.google.com/github/TYZAUM/Dogukan-DL-keras/blob/master/GooglePlayANN/Googleplayann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive/')
%cd /gdrive/
%cd 'My Drive/Colab-Notebooks'

In [0]:
import pandas as pd
import numpy as np
## data cleanup ##

df = pd.read_csv("googleplaystore.csv")
del df["Last Updated"]
size = df.iloc[:,4].values
installs = df.iloc[:,5].values
for i in range(len(size)):
    size[i] = size[i].replace("M","")
    size[i] = size[i].replace("k","") 
for i in range(len(installs)):
    installs[i] = installs[i].replace(",","")

x = df.iloc[:,1:5].join(df.iloc[:,6:10]).values
y = df["Installs"].values
#Label and Hot Encode
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.externals import joblib
from keras.utils import np_utils
categoryEncoder = LabelEncoder()
#ypeEncoder = LabelEncoder()
#contentEncoder = LabelEncoder()
#genreEncoder = LabelEncoder()
x[:,0] = categoryEncoder.fit_transform(x[:, 0])
x[:,4] = categoryEncoder.fit_transform(x[:,4])
x[:,6] = categoryEncoder.fit_transform(x[:,6])
x[:,7] = categoryEncoder.fit_transform(x[:,7])
oneHotEncoder = OneHotEncoder(categorical_features=[0,4,6,7])
x = oneHotEncoder.fit_transform(x).toarray()
joblib.dump(categoryEncoder,"encoder.pkl")

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,dummy_y, test_size=0.2,random_state = 0)

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
#%% NORMAL TRAIN
model =Sequential()
model.add(Dense(units = 120, init = 'uniform', activation = 'softsign', input_dim = 155))
model.add(Dense(units = 120, init = 'uniform', activation = 'softsign'))
model.add(Dense(units = 120, init = 'uniform', activation = 'softsign'))
model.add(Dense(units = 120, init = 'uniform', activation = 'softsign'))
model.add(Dense(units = 19, init = 'uniform', activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size = 100, epochs = 2000)